In [ ]:
%tensorflow_version 2.x
!pip install pydub &>/dev/null
!pip install mido &>/dev/null
!pip install pickle &>/dev/null

# Mount Drive

In [ ]:
from google.colab import drive
import subprocess
import os
import gc
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjRuBT0joHCWpz90ax23-LiwsU0zYJAa4rz9PUHOluneOmU_ZbH4UM


### MP3 files path

In [ ]:
pathmp3="/content/drive/My Drive/Colab Notebooks/mp3_splits/"
# out = subprocess.check_output(['ls',pathmp3]).decode('utf-8')
filesmp3 = os.listdir(pathmp3) 
# filesmp3

### MIDI files path

In [ ]:
pathmid="/content/drive/My Drive/Colab Notebooks/midi_splits/"
# out = subprocess.check_output(['ls',pathmid]).decode('utf-8')
filesmid = os.listdir(pathmid) 
# filesmid

In [ ]:
# sample_size = 18
# UseSavedata = False

data_path = '/content/drive/My Drive/Colab Notebooks/audio_data/'
gc.disable()

# Check if readed files are same





In [ ]:
print('mid',len(filesmid),'mp3',len(filesmp3))

mid 3635 mp3 3635


In [ ]:
filesmp3.sort()
filesmid.sort()
for i,j in zip(filesmid,filesmp3): 
  if not i[:-4] == j[:-4]:
    print(i," & ",j," is different!")


In [ ]:
import re
lossmid = []
lossmp3 = []
for i in filesmid:
  if not i[:-4]+'.mp3' in filesmp3:
    lossmid.append(i)

for i in filesmp3:
  if not i[:-4]+'.mid' in filesmid:
    lossmp3.append(i)

# len(extr)-len(filesmp3)

In [ ]:
print(len(lossmid),len(lossmp3))

0 0


### If there is loss, list the differences

In [ ]:

diffa =[ ]
diffb =[ ]

cmid = [row[1][:-4] for row in lossmid]
cmp3 = [row[1][:-4] for row in lossmp3]

list(set(cmid)-set(cmp3))

[]

## Toogle already readed data

In [ ]:
UseSavedata = True

# Import Library

In [ ]:
import warnings
import librosa
import IPython.display as ipd
from librosa import display as libdis
from mido import MidiFile
from pydub import AudioSegment

import numpy as np
from matplotlib import pyplot as plt


In [ ]:
import tensorflow as tf
import sklearn
import pickle
from time import sleep

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


---


# AudioFile Class for saving data

In [ ]:
class Audiofile:
  def __init__(self,x,sr,do_calc=False):
    self.x = x
    self.sr = sr
    if do_calc:
      self.stft = librosa.stft(x)
      self.spect = librosa.amplitude_to_db(abs(self.stft))
      self.mfccs = librosa.feature.mfcc(x, sr=sr)
      self.zero_crossings = librosa.zero_crossings(x, pad=False)
      self.sc = np.trim_zeros(librosa.feature.spectral_centroid(x, sr=sr)[0])
      self.sc_norm = sklearn.preprocessing.minmax_scale(self.sc, axis=0)
      frames = range(len(self.sc))
      self.time = librosa.frames_to_time(frames)
  
  def getSpectrogram(self,min_size):
    self.stft = librosa.stft(self.x[:min_size])
    self.spect = librosa.amplitude_to_db(abs(self.stft))
    return self.spect
  
  def getMfccs(self,min_size):
    self.mfccs = librosa.feature.mfcc(self.x[:min_size], sr=self.sr)
    return self.mfccs
  
  def getZerocrossings(self,min_size):
    self.zero_crossings = librosa.zero_crossings(self.x[:min_size], pad=False)
    return self.zero_crossings
  
  def getSpectralCentroid(self,min_size):
    self.sc = np.trim_zeros(librosa.feature.spectral_centroid(self.x[:min_size], sr=self.sr)[0])
    self.sc_norm = sklearn.preprocessing.minmax_scale(self.sc, axis=0)
    frames = range(len(self.sc))
    self.time = librosa.frames_to_time(frames)
    return self.sc_norm


# Read midi data as [***notes***]

In [ ]:
notes = []
if not UseSavedata:
  f = open(data_path+'note.dat','wb')  
  minnote_size = 99999999999999999999999999
  for i in filesmid:
    mid = MidiFile(pathmid+i, clip=True)
    note = []
    for msg in mid:
      # print(msg)
      if msg.type == 'note_on':
        # note.append(librosa.midi_to_note(msg.note))
        
        note.append([msg.channel,msg.note,msg.velocity,msg.time])
        # print(librosa.midi_to_note(msg.note))
    if len(note) < minnote_size:
      minnote_size = len(note)
    print("Shape of \'",i,'\':',len(note))
    notes.append(note)
  pickle.dump(notes,f)
  f.close()
else:
  f = open(data_path+'note.dat','rb')
  notes = pickle.load(f)
  f.close()

# Read mp3 data as [***audios***]

In [ ]:
warnings.filterwarnings('ignore')
audios = []  
minshape = 999999999999999999999
minindex = -1

if not UseSavedata:
  f = open(data_path+'audio.dat','wb')
  file_list = filesmp3
  for i in range(len(file_list)):
    x , sr = librosa.load(pathmp3+file_list[i])
    print(i+1,"- Shape of \'",file_list[i],'\':',x.shape)
    if x.shape[0]<minshape:
      minshape = x.shape[0]
      minindex = i
    audios.append(Audiofile(x,sr)) 
  pickle.dump([audios,minshape,minindex],f)
  f.close()
else:
  f = open(data_path+'audio.dat','rb')
  readed = pickle.load(f) 
  audios = readed[0]
  minshape = readed[1]
  minindex = readed[2]
  f.close()

warnings.filterwarnings('default')
gc.enable()

In [ ]:
print(np.array(audios).shape,np.array(notes).shape) 

(3635,) (3635,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


# Elimited all not unified size samples  

In [ ]:
mid_size = 100
mp3_size = 336640
mod_notes=[]
mod_audios=[]

for i,j in zip(audios,notes) :
  if len(j) == mid_size and len(i.x) == mp3_size:
    mod_notes.append(j)
    mod_audios.append(i)

In [ ]:
print(len(mod_audios),len(mod_notes))

3542 3542


# Cut in portion for validation set and training set

In [ ]:
cut_pos = int(len(mod_audios)*0.8)

# Genertate Feature sets 

In [ ]:
minshape = mp3_size
xs = []
mindata = 99999999999999
for i in mod_audios[:cut_pos]:
  # spect = i.getSpectrogram(minshape)

  mfcc = i.getMfccs(minshape).flatten()
  sc = i.getSpectralCentroid(minshape)
  zc = i.getZerocrossings(minshape)

  xs.append([mfcc,sc,zc])
  # xs.append(mfcc)

# print(mfcc.shape,sc.shape,zc.shape)
# train_x = np.array(xs)

xsv = []
for i in mod_audios[cut_pos:]:
  # spect = i.getSpectrogram(minshape)

  mfcc = i.getMfccs(minshape)
  sc = i.getSpectralCentroid(minshape)
  zc = i.getZerocrossings(minshape)
  xsv.append([mfcc,sc,zc])
  # xsv.append(mfcc)
  
# val_x = np.array(xsv)

In [ ]:
# print(train_x.shape,val_x.shape)
print(xs[:5])
print("====")
print(xsv[:5])

[[array([-752.9598   , -753.28156  , -753.31866  , ...,    1.4996172,
          1.5003318,    1.499563 ], dtype=float32), array([8.85401093e-01, 9.39023014e-01, 9.58087513e-01, 1.00000000e+00,
       5.73642147e-01, 7.93705479e-01, 8.10512889e-01, 7.77632569e-01,
       5.11718380e-01, 6.85935454e-01, 8.08336812e-01, 7.28919585e-01,
       4.78816781e-01, 1.86528631e-01, 6.67926271e-01, 7.40685768e-01,
       4.38841753e-01, 2.64873673e-02, 1.98188663e-02, 1.68929596e-02,
       1.55490941e-02, 1.68290052e-02, 2.52397309e-02, 4.66728373e-02,
       4.65425369e-02, 4.88466672e-02, 5.32820359e-02, 4.61976568e-02,
       3.61804474e-02, 3.86481233e-02, 4.35016954e-02, 3.96277177e-02,
       3.71817125e-02, 3.87911371e-02, 6.43727003e-02, 7.83027736e-02,
       7.28610136e-02, 6.67688749e-02, 6.53614188e-02, 6.45061614e-02,
       4.69012010e-02, 2.75807631e-02, 2.31445325e-02, 2.15230785e-02,
       2.43149629e-02, 2.30342525e-02, 1.83360906e-02, 1.52100331e-02,
       1.65063273e-02, 1.8

# Generate labels

In [ ]:
alt = [] 
minnote = mid_size
for i in mod_notes[:cut_pos]:
  alt.append(np.array(i,dtype=np.float))
train_y = np.array(alt)

alt = [] 
for i in mod_notes[cut_pos:]:
  alt.append(np.array(i,dtype=np.float))
val_y = np.array(alt)

In [ ]:
print(train_y.shape,val_y.shape)

(2833, 100, 4) (709, 100, 4)


# Duplicate data IF need

In [ ]:
amount = 1 #amount of duplication
r_x_train=[]
r_y_train=[]
r_x_val=[]
r_y_val=[]
for i in range(amount):
  for j in range(len(train_x)): 
    r_x_train.append(train_x[j])
    r_y_train.append(train_y[j])

for i in range(amount):
  for j in range(len(val_x)):    
    r_x_val.append(val_x[j])
    r_y_val.append(val_y[j])


dup_train_x = np.array(r_x_train,dtype="float64")
dup_train_y = np.array(r_y_train,dtype="float64")
dup_val_x = np.array(r_x_val,dtype="float64")
dup_val_y = np.array(r_y_val,dtype="float64")

In [ ]:
# print(train_x.shape,train_x)
print(train_x.shape,train_y.shape)
print('==================')
print(dup_train_x.shape,dup_train_y.shape)
print(dup_val_x.shape,dup_val_y.shape)
# plt.imshow(train_x)

(2833, 20, 658) (2833, 100, 4)
(2833, 20, 658) (2833, 100, 4)
(709, 20, 658) (709, 100, 4)


# Create Model

In [ ]:
class Autoencoder(Model):
  def __init__(self):
    super(Autoencoder, self).__init__()  
    self.encoder = tf.keras.Sequential([
      #mfcc
      layers.Input(shape=(train_x.shape[1],train_x.shape[2],1),name="input"), 
      
      #spectro centroid
      # layers.Input(shape=(train_x.shape[1],1),name="input"), 

      #------------
      #   mfcc
      #------------
      layers.Conv2D(32,kernel_size=(2,2),name="convo",
                        activation='relu'),
      layers.MaxPool2D(pool_size=2,strides=2, padding='valid',name="maxPool"),
      layers.Conv2D(64,kernel_size=(2,2),name="convo2",
                        activation='relu'),
      layers.MaxPool2D(pool_size=2,strides=2, padding='valid',name="maxPool2"),
      layers.Conv2D(128,kernel_size=(2,2),name="convo3",
                        activation='relu'),
      layers.MaxPool2D(pool_size=2,strides=2, padding='valid',name="maxPool3"),
      
      #------------
      #   Spectro Centroid
      #------------
      # layers.Conv1D(32,kernel_size=2,name="convo",
      #                   activation='relu'),
      # layers.MaxPool1D(pool_size=2,strides=2, padding='valid',name="maxPool"),
      # layers.Conv1D(64,kernel_size=2,name="convo2",
      #                   activation='relu'),
      # layers.Conv2D(128,kernel_size=(2,2),name="convo3",
      #                   activation='relu'),
      # layers.MaxPool2D(pool_size=2,strides=2, padding='valid',name="maxPool3"),
      #dense
      layers.Flatten(),
      layers.Dense(2048, activation='relu',name="dense-1"),#128
      layers.Dense(1024, activation='relu',name="dense-2"),#64
      layers.Dense(1024, activation='relu',name="dense-3"),#16
      layers.Dense(1024, activation='relu',name="dense-4")#8
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(int(train_y.shape[1]), activation='relu'),
      layers.Dense(train_y.shape[1]*train_y.shape[2],
                   activation='relu'),
      layers.Reshape((train_y.shape[1],train_y.shape[2]),
                     name="deco_output")
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
 
autoencoder = Autoencoder() 
autoencoder.build(train_x.shape)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
autoencoder.encoder.summary()
autoencoder.decoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 convo (Conv2D)              (None, 19, 657, 32)       160       
                                                                 
 maxPool (MaxPooling2D)      (None, 9, 328, 32)        0         
                                                                 
 convo2 (Conv2D)             (None, 8, 327, 64)        8256      
                                                                 
 maxPool2 (MaxPooling2D)     (None, 4, 163, 64)        0         
                                                                 
 convo3 (Conv2D)             (None, 3, 162, 128)       32896     
                                                                 
 maxPool3 (MaxPooling2D)     (None, 1, 81, 128)        0         
                                                                 
 flatten (Flatten)           (None, 10368)             0

# Train the model

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [ ]:
with tf.device('/device:GPU:0'):
  history = autoencoder.fit(dup_train_x, dup_train_y,
                  epochs=50,
                  shuffle=True,
                  validation_data=(dup_val_x, dup_val_y))

Epoch 1/50
89/89 [==============================] - 55s 615ms/step - loss: 639.6937 - val_loss: 628.9502
Epoch 2/50
89/89 [==============================] - 55s 619ms/step - loss: 628.2469 - val_loss: 616.8546
Epoch 3/50
89/89 [==============================] - 55s 620ms/step - loss: 617.8812 - val_loss: 618.9235
Epoch 4/50
89/89 [==============================] - 55s 620ms/step - loss: 615.1071 - val_loss: 606.3392
Epoch 5/50
89/89 [==============================] - 55s 616ms/step - loss: 601.5201 - val_loss: 582.5458
Epoch 6/50
89/89 [==============================] - 55s 621ms/step - loss: 582.6896 - val_loss: 580.3653
Epoch 7/50
89/89 [==============================] - 55s 619ms/step - loss: 582.3550 - val_loss: 592.1899
Epoch 8/50
89/89 [==============================] - 55s 618ms/step - loss: 580.1202 - val_loss: 571.9216
Epoch 9/50
89/89 [==============================] - 55s 618ms/step - loss: 569.2112 - val_loss: 568.1725
Epoch 10/50
89/89 [==============================] - 55

# Test the model


In [ ]:
output = "/content/drive/My Drive/Colab Notebooks/output/"

In [ ]:
x , sr = librosa.load(output+'canon-3.mp3')

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=63>
  if ffdec.available():
/usr/local/lib/python3.7/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=65>
  if ffdec.available():


In [ ]:

test_x_aud = Audiofile(x,sr)
mfcc = test_x_aud.getMfccs(minshape)
# sc = test_x_aud.getSpectralCentroid(minshape)
# sc

In [ ]:
# test = np.array(val_x[0]).reshape((val_x[0].shape[0],val_x[0].shape[1],1))
test = np.array(mfcc).reshape((mfcc.shape[0],mfcc.shape[1],1))
# test = np.array(val_x[0]).reshape((val_x[0].shape[0],1))
# test = sc.reshape((sc.shape[0],1))
test = np.array([test])
encoded = autoencoder.encoder(test).numpy()
decoded = autoencoder.decoder(encoded).numpy()
# decoded.tolist()[0] 

In [ ]:
import csv 

with open(output+"out.csv",'w',newline='') as f:
  cf = csv.writer(f)
  cf.writerows(np.round(decoded).tolist()[0])
f.close()